In [ ]:
# Import standard modules.

# Import supplemental modules.
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Notes on notation

# All citations are from:

# Tsyganenko et al, Ann. Geophys., 33, 1–11, 2015
# doi:10.5194/angeo-33-1-2015

# PpCcLl = page, column, line

# Notes on variable names:
# P = Solar wind dynamic pressure (P_dyn in paper), units of nP
# Bz = z-component of solar wind (B_z in paper), units of nT

In [ ]:
# Define the fitted values and uncertainties (Table 1)
# Uncertainties are RMS absolute deviations.
RH0, RH0_rms = 11.02, 0.05          
RH1, RH1_rms = 6.05, 0.88
RH2, RH2_rms = 0.84, 0.09
RH3, RH3_rms = -2.28, 0.08
RH4, RH4_rms = -0.25, 0.37
RH5, RH5_rms = -0.96, 0.16
T0, T0_rms = 0.29, 0.02
T1, T1_rms = 0.18, 0.08
a00, a00_rms = 2.91, 0.02
a01, a01_rms = -0.16, 0.07
a02, a02_rms = 0.56, 0.03
a10, a10_rms = 1.89, 0.03
a11, a11_rms = 0.06, 0.04
a12, a12_rms = 0.49, 0.04
alpha0, alpha0_rms = 7.13, 0.06
alpha1, alpha1_rms = 4.87, 0.07
alpha2, alpha2_rms = -0.22, 0.12
alpha3, alpha3_rms = -0.14, 0.04
chi, chi_rms = -0.29, 0.03  # 
beta0, beta0_rms = 2.18, 0.09
beta1, beta1_rms = 0.40, 0.11

# Constants for input scaling

# Pressure scale (nPa), P6C1L3
Pmean = 2.0

# Magnetic field scale (nT), P6C1L3
Bz0 = 5.0

In [ ]:
# Plot scaling functions for the input variables.
# The scaled values are the actual inputs to the model.

# Define the empirical equations for the input scaling.

# Note that chi < 0, so fP decreases with increasing P.
def fP_empirical(P):
    return (P/Pmean)**chi - 1

def fBz_empirical(Bz):
    return Bz/Bz0

# Figure parameters
figsize = (10, 5)
NROWS = 1
NCOLS = 2
MEAN_VALUE_COLOR = 'red'

# Set up figure for side-by-side plots.
fig = plt.figure(figsize=figsize)
fig.suptitle('Scaled input variables')
gs = mpl.gridspec.GridSpec(NROWS, NCOLS)

# Left plot: Pressure scaling
Pmin, Pmax, nP = 0, 4, 101
P = np.linspace(Pmin, Pmax, nP)[1:] # Skip 0
fPmin, fPmax = -1.0, 3.0
fP = fP_empirical(P)
ax = fig.add_subplot(gs[0])
ax.grid()
ax.set_xlabel("$P_{dyn}$ (nPa)")
ax.set_ylabel("$f_P$")
ax.set_title("Scaled solar wind dynamic pressure ($f_P$)")
ax.plot(P, fP)
# Plot vertical line at scale value.
ax.vlines(Pmean, fPmin, fPmax, color=MEAN_VALUE_COLOR)

# Right plot: Bz scaling
Bzmin, Bzmax, nBz = 0, 10, 101
Bz = np.linspace(Bzmin, Bzmax, nBz)
fBzmin, fBzmax = 0.0, 2.0
fBz = fBz_empirical(Bz)
ax = fig.add_subplot(gs[1])
ax.grid()
ax.set_xlabel("$B_z$ (nT)")
ax.set_ylabel("$f_{Bz}$")
ax.set_title("Scaled solar wind $B_z$ ($f_{Bz}$)")
ax.plot(Bz, fBz)
# Plot vertical line at scale value.
ax.vlines(Bz0, fBzmin, fBzmax, color=MEAN_VALUE_COLOR)

In [ ]:
# Define the empirical equations for the model parameters as a function of
# solar wind variables and coordinates.

# Model is in a cylindrical coordinate system centered on Earth.
# z = 0 corresponds to the GSM XY plane.
# rho = Geocentric distance (R_E)
# phi = Azimuthal angle (degrees)
# z = height above the GSM XY plane (R_E)

# fP = scaled pressure
# fBz = scaled Bz
# phi = azimuthal angle (radians)

In [ ]:
# Plot the empirical equation for RH.

# Define the empirical function.
def RH_empirical(fP, fBz, phi):
    return RH0 + RH1*fP + RH2*fBz + (RH3 + RH4*fP + RH5*fBz)*np.cos(phi)

# fP range
fPmin, fPmax, nfP = -1.0, 3.0, 21
fP = np.linspace(fPmin, fPmax, nfP)

# fBz range
fBzmin, fBzmax, nfBz = 0.0, 2.0, 41
fBz = np.linspace(fBzmin, fBzmax, nfBz)

# Azimuth angle (radians)
phi = 0.0

# Make a meshgrid for evaluation.
X, Y = np.meshgrid(fP, fBz)
# X and Y are shape (nfBz, nfP).
# The nfBz rows of X are the nfP values of fP.
# The nfP columns of Y are the nfBz values of fBz.
# X = X.T
# Y = Y.T
# X and Y are shape (nfP, nfBz).
# The nfP rows of X are the nfBz values of fBz.
# The nfBz columns of Y are the nfP values of fP.

# Compute the parameter at each point.
Z = RH_empirical(X, Y, phi)
# Shape is (nfBz, nfP)
# Z = Z.T
# Shape is (nfP, nfBz)

# Plot as image.
fig = plt.figure()
ax = fig.add_subplot()
aspect = nfP/nfBz
# aspect = 1
im = ax.imshow(Z, origin='lower', aspect=aspect)
ax.grid()

ax.set_xlabel("$f_P$")
ax.set_ylabel("$f_{Bz}$")

# Compute and show ticks and labels.
N_X_TICKS = 5
x_tick_pos = np.linspace(0, nfP - 1, N_X_TICKS)
x_tick_labels = ["%.1f" % (fPmin + x/(nfP - 1)*(fPmax - fPmin)) for x in x_tick_pos]
ax.set_xticks(x_tick_pos)
ax.set_xticklabels(x_tick_labels)

N_Y_TICKS = 5
y_tick_pos = np.linspace(0, nfBz - 1, N_Y_TICKS)
y_tick_labels = ["%.1f" % (fBzmin + y/(nfBz - 1)*(fBzmax - fBzmin)) for y in y_tick_pos]
ax.set_yticks(y_tick_pos)
ax.set_yticklabels(y_tick_labels)
ax.set_title("$R_H$ (empirical)")

fig.colorbar(im)

In [ ]:
print(X.shape, Y.shape, Z.shape)
print(RH_empirical(-1, 0, 0), X[0, 0], Y[0, 0], Z[0, 0])
print(RH_empirical(3, 0, 0), X[0, -1], Y[0, -1], Z[0, -1])
print(RH_empirical(3, 2, 0), X[-1, -1], Y[-1, -1], Z[-1, -1])
print(RH_empirical(-1, 2, 0), X[-1, 0], Y[-1, 0], Z[-1, 0])

In [ ]:
# Plot the empirical equation for T.

# Define the empirical function.
def T_empirical(fP, fBz, phi):
    return T0 + T1*fP

# Create the figure.
fig = plt.figure()

# fP range
fPmin, fPmax, nfP = -1.0, 3.0, 101
fP = np.linspace(fPmin, fPmax, nfP)

# Dummy array for fBz.
fBz = np.zeros_like(fP)

# Azimuth angle (radians)
phi = 0.0

# Compute the parameter at each point, then reorient.
T = T_empirical(fP, fBz, phi)

# Create the plot.
ax = plt.gca()
ax.plot(fP, T)
ax.grid()
ax.set_xlabel("$f_P$")
ax.set_ylabel("$T$")
ax.set_title("$T$ (empirical)")

In [ ]:
# Plot the empirical equation for a0.

# Define the empirical function.
def a0_empirical(fP, fBz, phi):
    return a00 + a01*fP + a02*fBz

# Create the figure.
fig = plt.figure()

# fP range
fPmin, fPmax, nfP = -1.0, 3.0, 21
fP = np.linspace(fPmin, fPmax, nfP)

# fBz range
fBzmin, fBzmax, nfBz = 0.0, 2.0, 41
fBz = np.linspace(fBzmin, fBzmax, nfBz)

# Azimuth angle (radians)
phi = 0.0

# Make a meshgrid for evaluation.
X, Y = np.meshgrid(fP, fBz)
# X and Y are shape (nfBz, nfP).
# The nfBz rows of X are the nfP values of fP.
# The nfP columns of Y are the nfBz values of fBz.
# X = X.T
# Y = Y.T
# X and Y are shape (nfP, nfBz).
# The nfP rows of X are the nfBz values of fBz.
# The nfBz columns of Y are the nfP values of fP.

# Compute the parameter at each point.
Z = a0_empirical(X, Y, phi)
# Shape is (nfBz, nfP)
# Z = Z.T
# Shape is (nfP, nfBz)

# Plot as image.
fig = plt.figure()
ax = fig.add_subplot()
aspect = nfP/nfBz
# aspect = 1
im = ax.imshow(Z, origin='lower', aspect=aspect)
ax.grid()

ax.set_xlabel("$f_P$")
ax.set_ylabel("$f_{Bz}$")

# Compute and show ticks and labels.
N_X_TICKS = 5
x_tick_pos = np.linspace(0, nfP - 1, N_X_TICKS)
x_tick_labels = ["%.1f" % (fPmin + x/(nfP - 1)*(fPmax - fPmin)) for x in x_tick_pos]
ax.set_xticks(x_tick_pos)
ax.set_xticklabels(x_tick_labels)

N_Y_TICKS = 5
y_tick_pos = np.linspace(0, nfBz - 1, N_Y_TICKS)
y_tick_labels = ["%.1f" % (fBzmin + y/(nfBz - 1)*(fBzmax - fBzmin)) for y in y_tick_pos]
ax.set_yticks(y_tick_pos)
ax.set_yticklabels(y_tick_labels)
ax.set_title("$a_0$ (empirical)")

fig.colorbar(im)

In [ ]:
print(X.shape, Y.shape, Z.shape)
print(a0_empirical(-1, 0, 0), X[0, 0], Y[0, 0], Z[0, 0])
print(a0_empirical(3, 0, 0), X[0, -1], Y[0, -1], Z[0, -1])
print(a0_empirical(3, 2, 0), X[-1, -1], Y[-1, -1], Z[-1, -1])
print(a0_empirical(-1, 2, 0), X[-1, 0], Y[-1, 0], Z[-1, 0])

In [ ]:
# Plot the empirical equation for a1.

# Define the empirical function.
def a1_empirical(fP, fBz, phi):
    return a10 + a11*fP + a12*fBz

# Create the figure.
fig = plt.figure()

# fP range
fPmin, fPmax, nfP = -1.0, 3.0, 21
fP = np.linspace(fPmin, fPmax, nfP)

# fBz range
fBzmin, fBzmax, nfBz = 0.0, 2.0, 41
fBz = np.linspace(fBzmin, fBzmax, nfBz)

# Azimuth angle (radians)
phi = 0.0

# Make a meshgrid for evaluation.
X, Y = np.meshgrid(fP, fBz)
# X and Y are shape (nfBz, nfP).
# The nfBz rows of X are the nfP values of fP.
# The nfP columns of Y are the nfBz values of fBz.
# X = X.T
# Y = Y.T
# X and Y are shape (nfP, nfBz).
# The nfP rows of X are the nfBz values of fBz.
# The nfBz columns of Y are the nfP values of fP.

# Compute the parameter at each point.
Z = a1_empirical(X, Y, phi)
# Shape is (nfBz, nfP)
# Z = Z.T
# Shape is (nfP, nfBz)

# Plot as image.
fig = plt.figure()
ax = fig.add_subplot()
aspect = nfP/nfBz
# aspect = 1
im = ax.imshow(Z, origin='lower', aspect=aspect)
ax.grid()

ax.set_xlabel("$f_P$")
ax.set_ylabel("$f_{Bz}$")

# Compute and show ticks and labels.
N_X_TICKS = 5
x_tick_pos = np.linspace(0, nfP - 1, N_X_TICKS)
x_tick_labels = ["%.1f" % (fPmin + x/(nfP - 1)*(fPmax - fPmin)) for x in x_tick_pos]
ax.set_xticks(x_tick_pos)
ax.set_xticklabels(x_tick_labels)

N_Y_TICKS = 5
y_tick_pos = np.linspace(0, nfBz - 1, N_Y_TICKS)
y_tick_labels = ["%.1f" % (fBzmin + y/(nfBz - 1)*(fBzmax - fBzmin)) for y in y_tick_pos]
ax.set_yticks(y_tick_pos)
ax.set_yticklabels(y_tick_labels)
ax.set_title("$a_1$ (empirical)")

fig.colorbar(im)

In [ ]:
print(X.shape, Y.shape, Z.shape)
print(a1_empirical(-1, 0, 0), X[0, 0], Y[0, 0], Z[0, 0])
print(a1_empirical(3, 0, 0), X[0, -1], Y[0, -1], Z[0, -1])
print(a1_empirical(3, 2, 0), X[-1, -1], Y[-1, -1], Z[-1, -1])
print(a1_empirical(-1, 2, 0), X[-1, 0], Y[-1, 0], Z[-1, 0])

In [ ]:
# Plot the empirical equation for alpha.

# Define the empirical function.
def alpha_empirical(fP, fBz, phi):
    return alpha0 + alpha1*np.cos(phi) + alpha2*fP + alpha3*fBz

# Create the figure.
fig = plt.figure()

# fP range
fPmin, fPmax, nfP = -1.0, 3.0, 21
fP = np.linspace(fPmin, fPmax, nfP)

# fBz range
fBzmin, fBzmax, nfBz = 0.0, 2.0, 41
fBz = np.linspace(fBzmin, fBzmax, nfBz)

# Azimuth angle (radians)
phi = 0.0

# Make a meshgrid for evaluation.
X, Y = np.meshgrid(fP, fBz)
# X and Y are shape (nfBz, nfP).
# The nfBz rows of X are the nfP values of fP.
# The nfP columns of Y are the nfBz values of fBz.
# X = X.T
# Y = Y.T
# X and Y are shape (nfP, nfBz).
# The nfP rows of X are the nfBz values of fBz.
# The nfBz columns of Y are the nfP values of fP.

# Compute the parameter at each point.
Z = alpha_empirical(X, Y, phi)
# Shape is (nfBz, nfP)
# Z = Z.T
# Shape is (nfP, nfBz)

# Plot as image.
fig = plt.figure()
ax = fig.add_subplot()
aspect = nfP/nfBz
# aspect = 1
im = ax.imshow(Z, origin='lower', aspect=aspect)
ax.grid()

ax.set_xlabel("$f_P$")
ax.set_ylabel("$f_{Bz}$")

# Compute and show ticks and labels.
N_X_TICKS = 5
x_tick_pos = np.linspace(0, nfP - 1, N_X_TICKS)
x_tick_labels = ["%.1f" % (fPmin + x/(nfP - 1)*(fPmax - fPmin)) for x in x_tick_pos]
ax.set_xticks(x_tick_pos)
ax.set_xticklabels(x_tick_labels)

N_Y_TICKS = 5
y_tick_pos = np.linspace(0, nfBz - 1, N_Y_TICKS)
y_tick_labels = ["%.1f" % (fBzmin + y/(nfBz - 1)*(fBzmax - fBzmin)) for y in y_tick_pos]
ax.set_yticks(y_tick_pos)
ax.set_yticklabels(y_tick_labels)
ax.set_title(r"$\alpha$ (empirical)")

fig.colorbar(im)

In [ ]:
print(X.shape, Y.shape, Z.shape)
print(alpha_empirical(-1, 0, 0), X[0, 0], Y[0, 0], Z[0, 0])
print(alpha_empirical(3, 0, 0), X[0, -1], Y[0, -1], Z[0, -1])
print(alpha_empirical(3, 2, 0), X[-1, -1], Y[-1, -1], Z[-1, -1])
print(alpha_empirical(-1, 2, 0), X[-1, 0], Y[-1, 0], Z[-1, 0])

In [ ]:
# Plot the empirical equation for beta.

# Define the empirical function.
def beta_empirical(fP, fBz, phi):
    return beta0 + beta1*fBz

# Create the figure.
fig = plt.figure()

# fBz range
fBzmin, fBzmax, nfBz = 0.0, 2.0, 101
fBz = np.linspace(fBzmin, fBzmax, nfBz)

# Dummy array for fP.
fP = np.zeros_like(fBz)

# Azimuth angle (radians)
phi = 0.0

# Compute the parameter at each point.
beta = beta_empirical(fP, fBz, phi)

# Create the plot.
ax = plt.gca()
ax.plot(fBz, beta)
ax.grid()
ax.set_xlabel("$f_{Bz}$")
ax.set_ylabel(r"$\beta$")
ax.set_title(r"$\beta$ (empirical)")